## Monitor locations for impacting events


### Connect to the portal

In [ ]:
from arcgis.gis import GIS, Item
from arcgis import features
from arcgis.features import GeoAccessor, GeoSeriesAccessor
gis = GIS("home") # If running outside of an ArcGIS Notebook this will need to be changed

### Setup - These should be the only variables you need to change

In [ ]:
# Ouptut Layer Name and Tag
output_item_title = 'Example Output Name' 
output_item_uniqueness_tag = 'example.unique.id'

# Item and layer IDs for the layer we are want to look for impacts to
# In this case, its data center locations that we're ETL'ing from Salesforce
monitor_item_id = ''
monitor_layer_id = 0 # Replace with the id of the layer you want to monitor for impacts to

# Item and layer IDs for the layer we want to check for impacts from
# In this example case, its the "Events ordered by size and severity" layer from the Living Atlass / weather.gov
impact_item_id = 'a6134ae01aad44c499d12feec782b386'
impact_layer_id = 5

#### Connect to the layers containing the locations we're monitoring and the data about what may be impacting them
Also search the portal to see if a layer with the correct output name & uniqueness tag exits

In [ ]:
# What we want to monitor
monitor_item = gis.content.get(monitor_item_id)
monitor_layer = monitor_item.layers[monitor_layer_id]

# What we want to look out for impacts from
impact_item = gis.content.get(impact_item_id)
impact_layer = impact_item.layers[impact_layer_id] # Get the "Events ordered by size and severity" layer

# What we will write results to 
# - This returns a list of items of len 0 - inf, handled by the output writing function
output_item_query = 'title:%s AND tags:%s' % (output_item_title, output_item_uniqueness_tag)
output_item_check = gis.content.search(query=output_item_query)

#### Define functions that will be used if we are updating an existing output layer

In [ ]:
def batch_it(l, n):
    for i in range(0, len(l), n):
        yield l[i:i + n]

def update_layer(target_feature_layer, updates_feature_set):
        """
        Push edits from SDF to hosted feature layer.
        """
        # Chunk edits into lists of 500 items. Python API can only push so many updates; item sized based on bytes.
        update_sets = list(batch_it(updates_feature_set.features, 25))
        for edits in update_sets:
            res = target_feature_layer.edit_features(adds=edits, rollback_on_failure=False)['addResults']
            print(f"Added {len([i for i in res if i['success']])} rows of {len(edits)}")

#### Check whether or not an output layer exists, perform the join, and write the outputs

In [ ]:
# If the search resulted in more than one item being returned, we don't have a way to guess which
# one we should be updating, so we'll just stop.
if len(output_item_check) > 1:
    print('Too many items were returned by the query.')
    print('Ensure that only one item has both the title "%s" and the tag "%s"' % (output_item_title, output_item_tag))
    print('Returned items')
    for item in [output_item_check]: print(' - ', item.homepage)
else:
    # Conditionally defining the result output name controls the behavior of spatial join.
    # - If there are no items in the list, result_output_name has a equal to output_item_title, and result will be a new Item
    # - If there are items in the list, result_output_name is None, and result will be a FeatureCollection.
    result_output_name = None if len(output_item_check) else output_item_title
    result = features.summarize_data.join_features(target_layer = impact_layer,
                                                   join_layer = monitor_layer,
                                                   spatial_relationship = 'intersects',
                                                   join_operation = 'JoinOneToMany',
                                                   output_name = result_output_name)
    if type(result) == Item:
        print('Initial Result Published, adding tags')
        result.update(item_properties={'tags': output_item_uniqueness_tag})
        print('Tags added')
    else: 
        print('Updating existing item')
        output_item = output_item_check[0]
        output_layer = output_item.layers[0]
        output_layer.manager.truncate()
        update_layer(output_layer, result.properties.featureSet)
        print('Updates complete')
print('See result at %s' % output_item.homepage)
        